In [1]:
from suffix_trees import STree
import suffix_tree
import pandas as pd

In [2]:
class STreeEx():
    def __init__(self, sTree):
        # 傳入原套件後綴樹
        self.sTree = sTree

    def lrs(self):
        # 最深非葉節點
        deepestNode = self._find_lrs(self.sTree.root)
        start = deepestNode.idx
        end = deepestNode.idx + deepestNode.depth
        return self.sTree.word[start:end]

    def _find_lrs(self, node):
        nodes = [self._find_lrs(n)
            for (n,_) in node.transition_links
            # 排除葉節點
            if n.transition_links != []]

        if nodes == []:
            return node
        
        deepestNode = max(nodes, key=lambda n: n.depth)
        return deepestNode

# Input File

In [3]:
input_file_path = "./input/TableA.txt"

In [4]:
content = []
recb_start = 0
recb_end = 0
tag = []
ids = []
classes = []
pathid = []
parentid = []
tecid = []
cecid = []
encoding = []
col = []
others = []
with open(input_file_path) as file:
    line = file.readline()
    while(line):
        line = file.readline()
        tmp = line.split('\t')
        if tmp[0] == "Content":
            content = tmp[1:]
        elif tmp[0] == "RecB":
            tmp = tmp[1:]
            for index in range(len(tmp)):
                if tmp[index] == "SDR":
                    recb_start = index
                elif tmp[index] == "EDR":
                    recb_end = index
        elif tmp[0] == "Tag":
            tag = tmp[1:]
        elif tmp[0] == "ID":
            ids = tmp[1:]
        elif tmp[0] == "Class":
            classes = tmp[1:]
        elif tmp[0] == "PathId":
            pathid = tmp[1:]
        elif tmp[0] == "ParentId":
            parentid = tmp[1:]
        elif tmp[0] == "TECId":
            tecid = tmp[1:]
        elif tmp[0] == "CECId":
            cecid = tmp[1:]
        elif tmp[0] == "Encoding":
            encoding = tmp[1:]
        elif tmp[0] == "Col Type":
            col = tmp[1:]
        else:
            others.append(tmp[1:])
others = others[0:-4]

# Translate node in to ascii code for suffix tree

In [5]:
encode = 0
whole_string = ""
node_dict = {} # code -> node num
code_dict = {} # code -> content
index_dict = {} # code -> first index
index = 0
for node in range(len(pathid)):
    if encode == 0:
        code = pathid[node] + "," + parentid[node] + "," + tecid[node] + "," + cecid[node] + "," + encoding[node] + "," + col[node]
    elif encode == 1:
        code = ids[node] + "," + classes[node] + "," + pathid[node] + "," + cecid[node]
    #if 'O' in col[node]: continue
    if code not in node_dict.keys():
        node_dict[code] = index
        code_dict[code] = others[0][node]
        index_dict[code] = node
        whole_string += chr(index)
        index += 1
    else:
        whole_string += chr(node_dict[code])
if encode == 0:
    print(pathid[0] + "," + parentid[0] + "," + tecid[0] + "," + cecid[0] + "," + encoding[0] + "," + col[0])
elif encode == 1:
    print(ids[0] + "," + classes[0] + "," + pathid[0] + "," + cecid[0])
inv_node_dict = {v: k for k, v in node_dict.items()} # node num -> code
inv_index_dict = {v: k for k, v in index_dict.items()} # index num -> code
print("\nConvert to ascii String:\n", whole_string)

0,"1:[1, 2]",0-2, ,1,MD

Convert to ascii String:
  	
 !"#$%&'()* !"#$%&'()*+,-.-/-0-1-2-3-4-5-6-7-8-9-:-;-<-=->-?-@-A-B-C-D-E-F-G-H-IJKLMNOPQRSIJKTUVPQRWIJKTMNOPQRWIJKTXNOPQRSIJKTXNYPQRWIJKTXNOPQRWIJKLXNOPQRWIJKTXNOPQRWIJKTZ[PQRSIJKT\NOPQRSIJKT\NYPQRWIJK]^OPQRW_`abcbdbebfghijklmnopqrstuvwxyz{|}~


In [6]:
def find_all_indexes(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1

# Longest repeat pattern

In [7]:
st = STree.STree(whole_string)
st = STreeEx(st)
lrs = str(st.lrs())
print("Length: ", len(lrs))
print(lrs)
for i in lrs:
    print("Content: ", code_dict[inv_node_dict[ord(i)]])

Length:  26
 !"#$%&'()*
Content:  Lowest price :: 29 :: 
Content:  # :: 30 :: 
Content:  All price :: 31 :: 
Content:  # :: 32 :: 
Content:  Free :: 33 :: 
Content:  # :: 32 :: 
Content:  TWD 1-1000 :: 35 :: 
Content:  # :: 32 :: 
Content:  TWD 1000-2000 :: 37 :: 
Content:  # :: 32 :: 
Content:  TWD 2000-3000 :: 39 :: 
Content:  # :: 32 :: 
Content:  Upon TWD 3000 :: 41 :: 
Content:  # :: 32 :: 
Content:  Time :: 43 :: 
Content:  # :: 30 :: 
Content:  All time :: 45 :: 
Content:  # :: 32 :: 
Content:  Today :: 47 :: 
Content:  # :: 32 :: 
Content:  In a week :: 49 :: 
Content:  # :: 32 :: 
Content:  In a month :: 51 :: 
Content:  # :: 32 :: 
Content:  In two months :: 53 :: 
Content:  # :: 32 :: 


# Show maximal repeats

Ignore maximal repeats if length equal to 1.

In [8]:
tree = suffix_tree.Tree({'A': whole_string})
print("Total Maximal repeats: ", len(tree.maximal_repeats()), '\n')
group = {}
top_repeats = (0, '')
for C, path in tree.maximal_repeats():
    if len(path) == 1: continue
    count = 0
    print("Length: ", len(path), "Pattern: {", path, "}")
    for id_, path2 in tree.find_all (path):
        count += 1
    print("Count: ", count)
    if count > top_repeats[0]:
        top_repeats = (count, str(path).replace(' ', ''))
    for i in path:
        print("Content: ", code_dict[inv_node_dict[ord(i)]])
    if index_dict[inv_node_dict[ord(path[0])]] not in group.keys():
        group[index_dict[inv_node_dict[ord(path[0])]]] = []
        group[index_dict[inv_node_dict[ord(path[0])]]].append(path)
    else:
        group[index_dict[inv_node_dict[ord(path[0])]]].append(path)
    print("\n", "="*50, "\n")

Total Maximal repeats:  32 

Length:  26 Pattern: {        !  "  #  $  %  &  '  (  )  *  }
Count:  2
Content:  Lowest price :: 29 :: 
Content:  # :: 30 :: 
Content:  All price :: 31 :: 
Content:  # :: 32 :: 
Content:  Free :: 33 :: 
Content:  # :: 32 :: 
Content:  TWD 1-1000 :: 35 :: 
Content:  # :: 32 :: 
Content:  TWD 1000-2000 :: 37 :: 
Content:  # :: 32 :: 
Content:  TWD 2000-3000 :: 39 :: 
Content:  # :: 32 :: 
Content:  Upon TWD 3000 :: 41 :: 
Content:  # :: 32 :: 
Content:  Time :: 43 :: 
Content:  # :: 30 :: 
Content:  All time :: 45 :: 
Content:  # :: 32 :: 
Content:  Today :: 47 :: 
Content:  # :: 32 :: 
Content:  In a week :: 49 :: 
Content:  # :: 32 :: 
Content:  In a month :: 51 :: 
Content:  # :: 32 :: 
Content:  In two months :: 53 :: 
Content:  # :: 32 :: 


Length:  3 Pattern: { I J K }
Count:  12
Content:  https://t.kfs.io/upload_images/105724/_____2019-11-26___4.50.59_medium.png :: 138 :: 
Content:  https://innovation.kktix.cc/events/vote-the-works-202

In [9]:
top_repeats

(12, 'IJK')

# Group by first element

In [10]:
group_num = 0
group_2 = {}
for index in group:
    print("Group ", group_num, " start element: ", code_dict[inv_index_dict[index]], " start index: ", index_dict[inv_index_dict[index]])
    print("Count: ", len(group[index]), "\n\n", "="*100, "\n")
    if len(group_2) == 0:
        group_2[index_dict[inv_index_dict[index]]] = []
        for path in group[index]:
            group_2[index_dict[inv_index_dict[index]]].append(path)
    else:
        check = 0
        for group_2_idx in group_2.keys():
            if index_dict[inv_index_dict[index]] - group_2_idx <= 36:
                for path in group[index]:
                    group_2[group_2_idx].append(path)
                check = 1
                break
        if not check:
            group_2[index_dict[inv_index_dict[index]]] = []
            for path in group[index]:
                group_2[index_dict[inv_index_dict[index]]].append(path)
    group_num += 1

Group  0  start element:  Lowest price :: 29 ::   start index:  29
Count:  1 


Group  1  start element:  https://t.kfs.io/upload_images/105724/_____2019-11-26___4.50.59_medium.png :: 138 ::   start index:  138
Count:  4 


Group  2  start element:  經過了三個禮拜的激戰，各方好手此時此刻正齊聚一堂，拿出最終實力一較高下 ; 除了實力的比拼以外，人氣指數也是選手們獲得獎勵的指標之一！話不多說，趕緊為你心中所屬的隊伍投下神聖的一票吧！ :: 142 ::   start index:  142
Count:  1 


Group  3  start element:  113 Attendees :: 143 ::   start index:  143
Count:  6 


Group  4  start element:  2020/1/7 :: 144 ::   start index:  144
Count:  2 


Group  5  start element:  ( :: 145 ::   start index:  145
Count:  8 


Group  6  start element:  Longboard初階課 課程時間：2 小時 課程費用：$700/人 初階真的很重要！因為你將學習正確的滑行動作及重心調整，並可享受到最真的滑行樂趣。 課程內容：方向性滑行、單雙腳橫向平衡訓練、腳煞、 :: 175 ::   start index:  174
Count:  4 




# Whole String split by top repeats

In [11]:
max_len = 0
seqs = []
pos = find_all_indexes(whole_string, top_repeats[1])
for idx in range(len(pos)):
    if idx+1 != len(pos):
        seqs.append(whole_string[pos[idx]:pos[idx+1]])
        if len(whole_string[pos[idx]:pos[idx+1]]) > max_len:
            max_len = len(whole_string[pos[idx]:pos[idx+1]])
    else:
        seqs.append(whole_string[pos[idx]:pos[idx]+max_len-1])
print(str(seqs))

['IJKLMNOPQRS', 'IJKTUVPQRW', 'IJKTMNOPQRW', 'IJKTXNOPQRS', 'IJKTXNYPQRW', 'IJKTXNOPQRW', 'IJKLXNOPQRW', 'IJKTXNOPQRW', 'IJKTZ[PQRS', 'IJKT\\NOPQRS', 'IJKT\\NYPQRW', 'IJK]^OPQRW']


# MSA

In [13]:
import cstar
scores = [1, -1, -4] # matchScore, mismatchScore, gapScore
msa = cstar.CenterStar(scores, seqs).msa()
trans_dict = {}
for i in range(len(msa)):
    if msa[i].replace('-', '') not in trans_dict.keys():
        trans_dict[msa[i].replace('-', '')] = msa[i]
    else: pass
for i in seqs:
    print(i, "\t-> ", trans_dict[i])
with open('./Set.txt', 'w', encoding='utf-8') as file:
    for page in range(len(others)):
        output_dict = {} # Record which pattern is used
        for s in range(len(seqs)):
            file.write(str(page) + "-0" + "-" + str(s) + "\t")
            tmp = find_all_indexes(whole_string, seqs[s])
            if seqs[s] not in output_dict.keys():
                output_dict[seqs[s]] = 0
            else:
                output_dict[seqs[s]] += 1
            #print("start:", tmp[output_dict[seqs[s]]], others[page][tmp[output_dict[seqs[s]]]])
            idx = 0
            for c in range(len(trans_dict[seqs[s]])):
                if trans_dict[seqs[s]][c] == '-':
                    file.write('\t')
                else:
                    file.write(others[page][tmp[output_dict[seqs[s]]]+idx] + "\t")
                    idx += 1
            file.write('\n')

IJKLMNOPQRS 	->  IJKLMNOPQRS
IJKTUVPQRW 	->  IJKT-UVPQRW
IJKTMNOPQRW 	->  IJKTMNOPQRW
IJKTXNOPQRS 	->  IJKTXNOPQRS
IJKTXNYPQRW 	->  IJKTXNYPQRW
IJKTXNOPQRW 	->  IJKTXNOPQRW
IJKLXNOPQRW 	->  IJKLXNOPQRW
IJKTXNOPQRW 	->  IJKTXNOPQRW
IJKTZ[PQRS 	->  IJKT-Z[PQRS
IJKT\NOPQRS 	->  IJKT\NOPQRS
IJKT\NYPQRW 	->  IJKT\NYPQRW
IJK]^OPQRW 	->  IJK-]^OPQRW
